This notebook is designed to directly run predictions from ts files on pngs, instead of converting pngs to dcms & then running from gen_rtr in mono repo.

In [4]:
from omegaconf import DictConfig
import importlib
import copy
from tqdm import tqdm

# import model_files
from qxr_utils.dicom.utils import get_array_from_dicom
from qxr_utils.image.transforms import scale, resize
from torch.utils.data import DataLoader
import torch.utils.data as data
import torch.nn.functional as fn


import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os

import torch


In [2]:
# import importlib
# importlib.reload(model_files)

# from model_files import *


In [2]:
test_df = pd.read_csv(
    "/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_updated_internal_test_13-08-24.csv",
      index_col=0)


In [4]:
# df.index.to_list()


In [3]:
images_dir = "/data_nas5/models/user_checkpoints/abhishek.rai/backup/projects/sbri_mass_testing/batch_2/dicoms/mass/pngs"


In [4]:
# folder_path = "/data_nas5/models/user_checkpoints/abhishek.rai/mass_v4_upgrade/fp_mass_testcases"


In [5]:
# with open('/home/users/abhishek.rai/My_notebooks/model/ribfracture_filenames.txt', 'r') as file:
#     filenames = file.read().splitlines()


In [6]:
class ScanD(data.Dataset):
    def __init__(self, imdict, imsize=960):
        self.imdict = imdict
        self.imsize = imsize
        self.indices = list(self.imdict.keys())

    def __getitem__(self, index):
        imid = self.indices[index]
        idx, im = self._get_im(imid)
        return {"id": idx, "input": im}

    def __len__(self):
        return len(self.indices)

    def _get_im(self, idx):
        try:
            impath = self.imdict[idx]
            ext = os.path.splitext(impath)[-1]
            if ext == ".dcm":
                try:
                    im = get_array_from_dicom(impath)
                except Exception as e:
                    #                     print(e)
                    im = None
            else:
                im = cv2.imread(impath, 0)
            if im is not None:
                im = resize(self.imsize, self.imsize)(im)
                im = scale(im)
                im = torch.Tensor(im).reshape(1, self.imsize, self.imsize)
                return idx, im
            else:
                return "none", torch.zeros(1, self.imsize, self.imsize)
        except Exception as e:
            print(e)
            print(idx)


In [7]:
def dict_to_df(preds):
    df = pd.DataFrame.from_dict(preds)
    df = df.transpose().reset_index()
    df = df.rename(columns={"index": "filename"})
    return df


In [8]:
# png_dict = {filename: os.path.join(images_dir, filename+".png") for filename in df.index.to_list() if os.path.exists(os.path.join(images_dir, filename+".png"))}
# png_dict = {filename: os.path.join(root, filename) for root, dirs, files in os.walk(images_dir) for filename in files}
# png_dict = {filename: os.path.join(images_dir, filename+".png") for filename in filenames if os.path.exists(os.path.join(images_dir, filename+".png"))}
png_dict = {filename.split(".png")[0]: os.path.join(images_dir, filename) for filename in os.listdir(images_dir) if filename.endswith(".png")}
dset = ScanD(png_dict, 960)


In [9]:
png_dict


{'3273dcf8-5f41bc6c-73c6f829-3fc51e72-a0e10503': '/data_nas5/models/user_checkpoints/abhishek.rai/backup/projects/sbri_mass_testing/batch_2/dicoms/mass/pngs/3273dcf8-5f41bc6c-73c6f829-3fc51e72-a0e10503.png',
 'b23d22f7-08a9ef0c-2e63e7ae-2b9e961e-3877eeb2': '/data_nas5/models/user_checkpoints/abhishek.rai/backup/projects/sbri_mass_testing/batch_2/dicoms/mass/pngs/b23d22f7-08a9ef0c-2e63e7ae-2b9e961e-3877eeb2.png',
 '9ee30fd2-4517f40f-3f6051f6-856a257a-cf06ea9c': '/data_nas5/models/user_checkpoints/abhishek.rai/backup/projects/sbri_mass_testing/batch_2/dicoms/mass/pngs/9ee30fd2-4517f40f-3f6051f6-856a257a-cf06ea9c.png',
 'eeea5096-ce10518e-3fda8609-8046b420-6aa31c23': '/data_nas5/models/user_checkpoints/abhishek.rai/backup/projects/sbri_mass_testing/batch_2/dicoms/mass/pngs/eeea5096-ce10518e-3fda8609-8046b420-6aa31c23.png',
 '55024b1f-9881b0a4-9934a99b-c14107eb-3c190071': '/data_nas5/models/user_checkpoints/abhishek.rai/backup/projects/sbri_mass_testing/batch_2/dicoms/mass/pngs/55024b1f-98

In [10]:
# def load_subset(dictionary, x):
#     return {k: dictionary[k] for k in list(dictionary)[:x]}


In [11]:
# png_dict_sub = load_subset(png_dict, 10)
# dset = model_files.ScanD(png_dict_sub, 960)


In [12]:
dset[0]['input'].shape

torch.Size([1, 960, 960])

In [13]:
len(dset)


73

In [14]:
dloader = DataLoader(dset, batch_size=16, num_workers=8)

In [15]:
def show_im(im_id, tag=None, which="im"):
    if which=="im":
        im_path = os.path.join(images_dir, im_id)
    elif which=="annot":
        im_path = os.path.join(annotation_dir, tag, im_id)
    
    im = cv2.imread(im_path, 0)
    plt.imshow(im, cmap='gray')
    
    
def get_tensor(im_id):
    im_path = os.path.join(images_dir, im_id)
    print(im_path)
    im = cv2.imread(im_path, 0)
    im = scale(resize(960, 960)(im))
    
    im = torch.tensor(im, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to("cuda:2")
    return im


In [16]:
model = torch.jit.load('/data_nas5/models/user_checkpoints/abhishek.rai/qureai/packages/python/qxr/traces_ts/v4_mass_cuda.ts', map_location='cuda')
# model.eval()


In [17]:
def run_model(model, dloader, names, seg=False, save_dir=None):
    
    if seg:
        assert save_dir, "save_dir not provided"
    
    preds = {}
    for batch in tqdm(dloader):
        with torch.no_grad():
            ids = batch['id']
            inputs = batch['input']
        
            out = model(inputs.to("cuda:0"))

            cls_out = out[0]
            seg_out = out[1]
            
            
            for tag in names:
                tdata = None
                sdata = None
                
                # if tag in cls_out:
                #     tdata = fn.softmax(cls_out[tag].cpu(), dim=1)[:,1]
                    
                if seg and (tag in seg_out):
                    sdata = fn.softmax(seg_out[tag].cpu(), dim=1)[:,1,:,:]
                    os.makedirs(os.path.join(save_dir, tag), exist_ok=True)

                tdata = cls_out[tag]
                sdata = seg_out[tag]

                for i, idx in enumerate(ids):
                    if not idx in preds:
                        preds[idx] = {}
                        
                    if not (tdata is None):
                        preds[idx].update({tag: tdata[i].item()})
                        
                    if not (sdata is None):
                        s_out = sdata[i].detach().cpu().numpy()
                        seg_max = s_out.max().item()
                        
                        preds[idx].update({f"{tag}_seg": seg_max})
                        cv2.imwrite(f"{save_dir}/{tag}/{idx}.png", (s_out)*255)   
                        
                        
    
    return dict_to_df(preds)


In [18]:
out = run_model(model, dloader, #['endotracheal_tube_cls','nasogastric_tube_cls','tracheostomy_tube_cls','central_venous_catheter_tip_cls'],
                                 ['mass','mediastinal_mass'],
                                  seg=False, save_dir="")


100%|██████████| 5/5 [00:19<00:00,  3.83s/it]


In [19]:
out


,filename,mass,mass_seg,mediastinal_mass,mediastinal_mass_seg
0,3273dcf8-5f41bc6c-73c6f829-3fc51e72-a0e10503,4.308840e-05,0.002474,0.151091,0.066941
1,b23d22f7-08a9ef0c-2e63e7ae-2b9e961e-3877eeb2,7.768479e-03,0.005317,0.000011,0.000351
2,9ee30fd2-4517f40f-3f6051f6-856a257a-cf06ea9c,1.780066e-01,0.458210,0.000210,0.001038
3,eeea5096-ce10518e-3fda8609-8046b420-6aa31c23,2.104375e-07,0.000554,0.000006,0.000326
4,55024b1f-9881b0a4-9934a99b-c14107eb-3c190071,1.018091e-01,0.762217,0.000094,0.000996
...,...,...,...,...,...
68,35d8e7fa-64192ea3-468c7257-76ab5fda-c934b78f,1.342975e-04,0.002843,0.000183,0.000681
69,5f9b2aae-04b97ee9-e71c7ca8-a6696533-d400eb70,1.042808e-02,0.033077,0.000102,0.000815
70,940a9c38-3c46c6c7-d5e4c022-4eeaeb3a-e162916e,8.372279e-05,0.003152,0.000006,0.000494
71,073633cf-5a5ddda3-e77a0d6d-5bec5ae4-195a5d41,1.318888e-03,0.056638,0.000995,0.005184


In [25]:
out['mass_avg'] = out[['mass', 'mass_seg']].mean(axis=1)
out['mediastinal_mass_avg'] = out[['mediastinal_mass', 'mediastinal_mass_seg']].mean(axis=1)


In [ ]:
out['mass_avg']


In [26]:
out['mass_th'] = 0.5


In [29]:
out['mass_pred'] = out['mass_avg']>out['mass_th']
out['mass_pred'].value_counts()


False    66
True      7
Name: mass_pred, dtype: int64

In [29]:
df_ = pd.read_csv('/data_nas5/models/user_checkpoints/abhishek.rai/tubes/discordant_cases/ensembled_discordants.csv')
df_

,filename,cvc,cvc_seg,ett,ett_seg,ngt,ngt_seg,tst,tst_seg,ca_junction_seg,carina_seg,ge_junction_seg
0,1.2.392.200036.9125.9.0.3827062279.201334784.4...,0.839997,0.229564,0.902936,0.976249,0.050718,0.235887,0.000301,0.016481,0.261661,0.396308,0.360831
1,1.2.840.113564.101037108.2019042813562815692.2...,0.294064,0.270543,0.600423,0.649254,0.047659,0.135116,0.001217,0.026955,0.160517,0.181411,0.103649
2,1.2.840.113564.101037108.2020040200182835932.2...,0.125106,0.185917,0.801957,0.953692,0.139789,0.098321,0.020542,0.070726,0.258791,0.387690,0.080401
3,1.2.840.113564.101037108.2020050503120034323.2...,0.016116,0.049928,0.591601,0.586849,0.037994,0.050890,0.031151,0.074452,0.135088,0.104418,0.049319
4,esophagealatresiatef04mdnasogastrictubecoiledi...,0.051411,0.034557,0.024583,0.193046,0.199866,0.292335,0.000052,0.013796,0.299180,0.283582,0.271533
5,malpositioned-nasogastric-tube-6 ...,0.964786,0.358188,0.763829,0.850025,0.479343,0.528655,0.002304,0.029037,0.414213,0.553742,0.376391
6,nasogastrictubemalfunction04mdnasogastrictubel...,0.091848,0.057803,0.076101,0.448315,0.386027,0.550409,0.000436,0.014891,0.263922,0.444540,0.253929


In [32]:
(0.902936+0.976249)/2


0.9395925

In [ ]:
out.to_csv('/data_nas5/models/user_checkpoints/abhishek.rai/v4_segmentations/pred_scores_v4_ribfracture_positives.csv', index = False)